In [20]:
import OpenDartReader
import pandas as pd
from datetime import datetime
import time
import warnings

warnings.simplefilter("ignore")
api_key = '08d5ae18b24d9a11b7fd67fb0d79c607f1c88464'
dart = OpenDartReader(api_key)
today = datetime.today().strftime("%Y%m%d") #오늘

#code : 종목코드
#corp : 종목이름
#fs_div : 분기 1(11013),2(11012),3(11014),4(11011)
#reprt_code : 연결, 개별 ( OFC, CFS)

def quater_fin(code,corp,year,reprt_code,fs_div='CFS'):
    # try:
    df = dart.finstate_all(code, year,reprt_code=reprt_code,fs_div=fs_div)
    
    df = df[df['sj_nm'].str.contains('손익')]
    
    df = df[df['account_id'].str.contains('ifrs-full_Revenue')|df['account_id'].str.contains('OperatingIncomeLoss')|df['account_nm'].str.contains('영업수익')]

    df = df[['account_nm','thstrm_amount']]
    
    if reprt_code == '11014': # 3분기
        df.columns = ['항목',f'{year}.3Q']
        df = df.apply(pd.to_numeric,errors = 'ignore')
        df = df.sort_values(by='항목',ascending=True)
    
        df['항목'].iloc[0] = '매출액'
        df['항목'].iloc[1] = '영업이익'
        df.insert(0,'기업',corp)
        df = df.set_index(['기업','항목'])
        df = df.apply(lambda x : round(x/100000000,0))
        
        sales = df.loc[(corp, '매출액'), f'{year}.3Q']
        op = df.loc[(corp, '영업이익'), f'{year}.3Q']
        opm = (op / sales) * 100  # 백분율로 변환
        df.loc[(corp, 'OPM'), f'{year}.3Q'] = opm
        df = df.apply(lambda x : round(x,1))  
        
        
    elif reprt_code == '11012': #반기
        df.columns = ['항목',f'{year}.2Q']
        df = df.apply(pd.to_numeric,errors = 'ignore')
        df = df.sort_values(by='항목',ascending=True)
        df['항목'].iloc[0] = '매출액'
        df['항목'].iloc[1] = '영업이익'
        df.insert(0,'기업',corp)
        df = df.set_index(['기업','항목'])
        df = df.apply(lambda x : round(x/100000000,0))   
        
        sales = df.loc[(corp, '매출액'), f'{year}.2Q']
        op = df.loc[(corp, '영업이익'), f'{year}.2Q']
        opm = (op / sales) * 100  # 백분율로 변환
        df.loc[(corp, 'OPM'), f'{year}.2Q'] = opm
        df = df.apply(lambda x : round(x,1))  
         
         
    elif reprt_code == '11013': # 1분기
        df.columns = ['항목',f'{year}.1Q']
        df = df.apply(pd.to_numeric,errors = 'ignore')
        df = df.sort_values(by='항목',ascending=True)
        df['항목'].iloc[0] = '매출액'
        df['항목'].iloc[1] = '영업이익'
        df.insert(0,'기업',corp)
        df = df.set_index(['기업','항목'])
        df = df.apply(lambda x : round(x/100000000,1))  
        
        sales = df.loc[(corp, '매출액'), f'{year}.1Q']
        op = df.loc[(corp, '영업이익'), f'{year}.1Q']
        opm = (op / sales) * 100  # 백분율로 변환
        df.loc[(corp, 'OPM'), f'{year}.1Q'] = opm
        df = df.apply(lambda x : round(x,1))  
       
       
    elif reprt_code == '11011': # 사업보고서
        df.columns = ['항목',f'{year}.4Q']
        df = df.apply(pd.to_numeric,errors = 'ignore')
        df = df.sort_values(by='항목',ascending=True)
        df['항목'].iloc[0] = '매출액'
        df['항목'].iloc[1] = '영업이익'
        df.insert(0,'기업',corp)
        df = df.set_index(['기업','항목'])
        df = df.apply(lambda x : round(x/100000000,0))    

        sales = df.loc[(corp, '매출액'), f'{year}.4Q']
        op = df.loc[(corp, '영업이익'), f'{year}.4Q']
        opm = (op / sales) * 100  # 백분율로 변환
        df.loc[(corp, 'OPM'), f'{year}.4Q'] = opm
        df = df.apply(lambda x : round(x,1))  

    # if fs_div == 'CFS':
    #     df.insert(0,'구분','연결')
        
    # else: 
    #     df.insert(0,'구분','별도')
    # except Exception as e:
    #     print(corp,e)
    time.sleep(2)
    return df

def year_finance(result_df):
    year_df = result_df[result_df.columns.str.contains('4Q')]
    return year_df
    





#fs_div : 분기 1(11013),2(11012),3(11014),4(11011)
code = '097230'
corp = 'HJ중공업'
years = ['2020','2021','2022','2023','2024']
# years = ['2023'] # 평가

reprt_codes = ['11013','11012','11014','11011']
# reprt_codes = ['11014'] # 평가

fs_div = 'CFS' # OFS 개별, CFS 연결

result_df = pd.DataFrame()
for year in years:
    for reprt_code in reprt_codes:
        try:
            df = quater_fin(code,corp,year,reprt_code,fs_div)
            
            result_df = pd.concat([result_df,df],axis=1)
        except Exception as e:
            print(e)
# result_df.to_clipboard()
# result_df.to_excel(f'./개별종목_재무/{corp}_{today}.xlsx')
result_df
# a = year_finance(result_df)
# a

reprt_code='11013', fs_div='CFS' (1분기보고서, 연결제무제표)'
reprt_code='11012', fs_div='CFS' (반기보고서, 연결제무제표)'
reprt_code='11014', fs_div='CFS' (3분기보고서, 연결제무제표)'
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
reprt_code='11013', fs_div='CFS' (1분기보고서, 연결제무제표)'
reprt_code='11012', fs_div='CFS' (반기보고서, 연결제무제표)'
reprt_code='11014', fs_div='CFS' (3분기보고서, 연결제무제표)'
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
reprt_code='11013', fs_div='CFS' (1분기보고서, 연결제무제표)'
reprt_code='11012', fs_div='CFS' (반기보고서, 연결제무제표)'
reprt_code='11014', fs_div='CFS' (3분기보고서, 연결제무제표)'
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
reprt_code='11013', fs_div='CFS' (1분기보고서, 연결제무제표)'
reprt_code='11012', fs_div='CFS' (반기보고서, 연결제무제표)'
reprt_code='11014', fs_div='CFS' (3분기보고서, 연결제무제표)'
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
reprt_code='11013', fs_div='CFS' (1분기보고서, 연결제무제표)'
reprt_code='11012', fs_div='CFS' (반기보고서, 연결제무제표)'
reprt_code='11014', fs_div='CFS' (3분기보고서, 연결제무제표)'
reprt_code='11011', fs_div='CFS' (사업보고서,

2020.1Q  2020.2Q  2020.3Q  2020.4Q  2021.1Q  2021.2Q  2021.3Q  \
기업    항목                                                                    
HJ중공업 매출액    3353.4   4904.0   4710.0  16958.0   2744.8   4410.0   4777.0   
      영업이익    171.0      4.0    445.0    516.0   -251.6     10.0     82.0   
      OPM       5.1      0.1      9.4      3.0     -9.2      0.2      1.7   

            2021.4Q  2022.1Q  2022.2Q  2022.3Q  2022.4Q  2023.1Q  2023.2Q  \
기업    항목                                                                    
HJ중공업 매출액   17065.0   3479.9   4455.0   3978.0  17882.0   4050.2   5014.0   
      영업이익  -1090.0     25.8     23.0     -0.0     66.0   -110.9   -752.0   
      OPM      -6.4      0.7      0.5     -0.0      0.4     -2.7    -15.0   

            2023.3Q  2023.4Q  2024.1Q  2024.2Q  2024.3Q  
기업    항목                                                 
HJ중공업 매출액    4905.0  21621.0   4675.8   4848.0   4538.0  
      영업이익   -409.0  -1088.0    119.0   -395.0    329.0  
      OPM      -8.3     -5.0      2.5     -8.1      7.2

In [19]:
result_df.to_clipboard()